# INTRODUCCIÓN

Importamos las librerías necesarias para el estudio de este caso práctico.

In [ ]:
#Importaciones básicas: librerías y otros aspectos

import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from string import ascii_letters
from sklearn.model_selection import train_test_split

from sklearn.dummy import DummyRegressor

from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

#Importaciones Para KNN

from joblib import dump
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsClassifier

#Para árboles

from sklearn import tree

#PCA

from sklearn.decomposition import PCA




Lo primero de todo será importar los datos que necesitamos para poder trabajar con ellos.


In [ ]:
#Carga de los datos  de estudio

disp_df = pd.read_csv("disp_st4ns1.txt", index_col=0)

#Opciones de display para que nos muestre la tabla entera

pd.options.display.max_columns = 100
pd.options.display.max_rows = 5000

#Display del dataset

#disp_df

#ANALISIS EXPLORATORIO DE DATOS (EDA)


En el análisis exploratorio de datos, procederemos a analizar brevemente el conjunto de datos proporcionado de la siguiente manera: COMPLETAR

Para empezar, comprobamos que no existen missing values en el conjunto de datos:

In [ ]:
print(disp_df.isnull().values.sum())

0


Después, vamos a mostrar algunos parámetros de la tabla: media, std..etc.

In [ ]:
disp_df.describe()

Lo primero, analizar las variables de manera individual: al haber 75 features divididos en 15 variables meteorológicas, comprobaremos una a una si son constantes o no aportan información. Si lo son, procederemos a eliminarlas.

In [ ]:
describe_var = disp_df.describe()

max_values = describe_var.loc['max']
min_values = describe_var.loc['min']

cols_to_drop = []
for column_name, max_value in max_values.iteritems():
  if max_value == min_values[column_name]:
    cols_to_drop.append(column_name)

describe_var = describe_var.drop(cols_to_drop, axis=1)

print(describe_var)

En nuestro caso, no se ha eliminado ninguna variable al todas, por mínimas que sean, aportan información.
((tabla de maximos y minimos????)

Ahora, vamos a crear una matriz de correlación para ver la similitud entre pares de variables. También, lo comprobaremos de manera gráfica con un heatmap:



In [ ]:
disp_df.corr()

In [ ]:
sns.set_theme(style="white")

median = np.median(disp_df)

corr_matrix = disp_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(76, 76))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_matrix, annot = True, mask=mask, cmap=cmap, vmin = -1, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})


# SEPARACIÓN DE LOS DATOS



Ahora vamos a separar el data set en el conjunto de train y test dividido por años, en concreto los 10 primeros y los dos últimos años:



In [ ]:
X = disp_df.iloc[:, 0:75]
y = disp_df.iloc[:, 75]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=2/12, shuffle = False)

print(X_train.shape, y_train.shape)

(3650, 75) (3650,)


#METODOS BÁSICOS

Los tres métodos básicos que se piden en esta práctica son: Vecinos Más Cercanos (KNN o K  Nearest Neighbors), Árboles de decisión y regresión lineal. 

In [ ]:
def rmse (y_test, y_test_pred):
  """ Computation of root mean squared error """
  return np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

def mae (y_test, y_test_pred):
  """ Computation of mean absolute error """
  return metrics.mean_absolute_error(y_test, y_test_pred)



Antes de empezar a realizarlos, vamos a crear 2 modelos regresores "dummy" utilizando tres métodos estadísticos distintos. Así, podremos comparar su eficiencia con los modelos que vayamos realizando más tarde.

In [ ]:
dummy_mean = DummyRegressor()
dummy_mean.fit(X_train, y_train)
dummy_mean_pred = dummy_mean.predict(X_test)

dummy_median = DummyRegressor(strategy='median')
dummy_median.fit(X_train, y_train)
dummy_median_pred = dummy_median.predict(X_test)


print("MAE del Dummy media:", mae(y_test,dummy_mean_pred))
print("RMSE del Dummy mediana:", rmse(y_test,dummy_median_pred))

MAE del Dummy media: 6490928.170088197
RMSE del Dummy mediana: 7646362.522614248


## KNN

El primer método que vamos a realizar es el de Vecinos más cercanos (KNN). Vamos a seguir unos pasos claros a la hora de crear, probar y ajustar el modelo:

*   Crear el modelo y entrenarlo para ver su resultado genérico
*   Utilizar el Predefinedsplit para hacer una Validación Cruzada sin desordenar los datos
*   Ajustar el hiperparámetro usando el PredefinedSplit calculado anteriormente
*   Volver a entrenar el modelo con el parámetro optimizado

Al no poder desordenar los datos debido a la no-aleatoriedad de los mismos (están ordenados por años) no se podrá usar el Cross Validation convencional en ningún modelo, aunque si utilizaremos el predefined split para ajustar los hipérparametros.



Antes de nada, vamos a normalizar los datos y dejarlos así para el resto de modelos:

In [ ]:
# Normalización de los datos

scaler = MinMaxScaler()
X_train_normalizado = scaler.fit_transform(X_train)
X_test_normalizado = scaler.transform(X_test)

array([[0.        , 0.        , 0.        , ..., 0.24962631, 0.232826  ,
        0.22757815],
       [0.        , 0.        , 0.        , ..., 0.1758844 , 0.15425751,
        0.14402571],
       [0.        , 0.        , 0.        , ..., 0.25610364, 0.24084739,
        0.23371312],
       ...,
       [0.03744549, 0.00439498, 0.0065114 , ..., 0.17787743, 0.21184698,
        0.20858896],
       [0.        , 0.        , 0.        , ..., 0.21026408, 0.2143151 ,
        0.19661116],
       [0.        , 0.        , 0.        , ..., 0.26606876, 0.24372686,
        0.23867952]])

También vamos a dejar las particiones del predefinedsplit hechas

In [ ]:
N_train = 7 * 365
N_valid = 3 * 365

selector = [-1] * N_train + [0] * N_valid
inner = PredefinedSplit(selector)

inner.get_n_splits(X_train)

for train, valid in inner.split(X_train):
  print(f"Size of the train set: {train.shape}")
  print(f"Size of the valid set: {valid.shape}")


Size of the train set: (2555,)
Size of the valid set: (1095,)
Indices of the train set: [   0    1    2 ... 2552 2553 2554]
Indices of the valid set: [2555 2556 2557 ... 3647 3648 3649]


Creamos el modelo, lo fiteamos y computamos su acurracy con el hiperparámetro por defecto:

In [ ]:
# Crear el modelo
Knn_model= KNeighborsClassifier()

# Fiteamos el modelo
start_knn = time.time()
Knn_model.fit(X_train_normalizado, y_train)

# Computamos la predicción del modelo
y_prediccion_knn = Knn_model.predict(X_test_normalizado)

end_knn = time.time()

print("MAE del KNN:", mae(y_test,y_prediccion_knn))
print("RMSE del KNN:", rmse(y_test,y_prediccion_knn))
print("Tiempo de ejecución para entrenar el modelo: ", end_knn - start_knn)


AttributeError: ignored

Encontrar el mejor hiperparámetro con GridSearch

In [ ]:
#No pares para evitar empates
parametros_a_ajustar_knn = {
    'n_neighbors': [3,5,7,9,11,13,15],
    'weights': ['uniform', 'distance'],
    'p': [1,2]
}

Knn_model_ajust = KNeighborsClassifier()

grid_search_knn = GridSearchCV(Knn_model_ajust, param_grid = parametros_a_ajustar_knn, cv = inner, scoring ='neg_mean_squared_error')

start_knn_ajust = time.time()

grid_search_knn.fit(X_train_normalizado, y_train)

mejor_modelo_KNN = grid_search_knn.best_estimator_

y_prediccion_ajustado = mejor_modelo_KNN.predict(X_test_normalizado)

end_knn_ajust = time.time()

Ahora, ya teniendo los hiperparámetros en orden, solo queda volver a entrenar el modelo y ver su accuracy, esta vez correctamente:

In [ ]:
print("MAE del KNN ajustado:", mae(y_test,y_prediccion_ajustado))
print("RMSE del KNN ajustado:", rmse(y_test,y_prediccion_ajustado))
print("Tiempo de ejecución para entrenar el modelo: ", end_knn_ajust - start_knn_ajust)

MAE del KNN ajustado: 2985800.5479452056
RMSE del KNN ajustado: 4307815.449427792
Tiempo de ejecución para entrenar el modelo:  7.800419092178345


Opcional: encontrar el mejor hiperparámetro por fuerza bruta, graficos comparando tiempo de ejecución??

## ÁRBOLES DE REGRESIÓN

Con los árboles vamos a seguir exactamente el mismo proceso que con Knn: crear el modelo, ajustar hiperparámetros con PredefinedSplit y GridSearchCV, y luego volver a entrenarlo:

In [ ]:
arbol_decision = tree.DecisionTreeClassifier()

start_arbol = time.time()

arbol_decision.fit(X_train_normalizado, y_train)

y_prediccion_arbol = arbol_decision.predict(X_test_normalizado)

end_arbol = time.time()

print("MAE del árbol de decisión ajustado:", mae(y_test,y_prediccion_arbol))
print("RMSE del árbol de decisión ajustado::", rmse(y_test,y_prediccion_arbol))
print("Tiempo de ejecución para entrenar el modelo: ", end_arbol - start_arbol)


MAE del árbol de decisión ajustado: 3569952.7397260275
RMSE del árbol de decisión ajustado:: 5036636.232587277
Tiempo de ejecución para entrenar el modelo:  48.347015142440796


Gridsearch: esto está bien??????

In [ ]:
parametros_a_ajustar_arbol = {
    'max_depth': range(2, 16, 2),
    'min_samples_split': range(2, 16, 2)
}

arbol_dec_ajust = tree.DecisionTreeClassifier()

grid_search_arbol = GridSearchCV( arbol_dec_ajust, param_grid = parametros_a_ajustar_arbol, cv = inner, scoring='neg_mean_squared_error')

start_arbol_ajust = time.time()

grid_search_arbol.fit(X_train_normalizado, y_train)

mejor_arbol = grid_search_arbol.best_estimator_

y_prediccion_ajustado_arbol = grid_search_arbol.predict(X_test_normalizado)

end_arbol_ajust = time.time()


KeyboardInterrupt: ignored

Entrenamiento del modelo final del árbol de decisión:

In [ ]:
#mejor_modelo_arbol = grid_search_arbol.best_estimator_

print("MAE del árbol de decisión ajustado:", mae(y_test,y_prediccion_ajustado_arbol))
print("RMSE del árbol de decisión ajustado::", rmse(y_test,y_prediccion_ajustado_arbol))
print("Tiempo de ejecución para entrenar el modelo: ", end_arbol - start_arbol)

#y_prediccion_ajustado_arbol = mejor_modelo_arbol.predict(X_test_normalizado)


MAE del árbol de decisión ajustado: 8234616.164383561
RMSE del árbol de decisión ajustado:: 10077144.805963451
Tiempo de ejecución para entrenar el modelo:  48.347015142440796


## REGRESIÓN LINEAL

## CONCLUSIONES SOBRE MÉTODOS BÁSICOS

# REDUCIÓN DE LA DIMENSIONALIDAD

In [ ]:
#Datos normalizados al principio del proyecto

pca = PCA(0.95)
pca.fit(X_train_normalizado)
X_train_pca = pca.transform(X_train_normalizado)
X_test_pca = pca.transform(X_test_normalizado)

print(X_train_pca.shape, X_test_pca.shape)


(3650, 8) (730, 8)


Probar los modelos de nuevo ahora con los PCA

KNN

In [ ]:
grid_search_knn.fit(X_train_normalizado, y_train)

mejor_modelo_KNN = grid_search_knn.best_estimator_

y_prediccion_ajustado = mejor_modelo_KNN.predict(X_test_normalizado)

ARBOLES

In [ ]:
"""scaler = RobustScaler()
pca = PCA(n_components=1)
lr = LinearRegression()
pipe_scale_pca_lr = Pipeline([
  ('scale', scaler),
  ('pca', pca),
  ('lr', LinearRegression)])
pipe_scale_pca_lr.fit(X_train)
preds_test = pipe_scale_pca_knn.predict(X_test)


# define pipeline with PCA and linear regression
lr_pca = Pipeline([('scaler', RobustScaler()), ('pca', PCA(0.85)), ('lr', LinearRegression())])

start_time = time.time()
lr_pca.fit(X_train, y_train)
end_time = time.time()
elapsed_time_lr_pca = end_time - start_time

lr_pca_pred = lr_pca.predict(X_test)"""




"""scaler = RobustScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.fit_transform(X_test)
pca = PCA(0.85)
pca.fit(X_train_norm)
X_train_pca_norm = pca.transform(X_train_norm)
X_test_pca_norm = pca.transform(X_test_norm)
lr_default_pca = LinearRegression()
lr_default_pca.fit(X_train_pca_norm, y_train)
lr_default_pca.score(X_train_pca_norm, y_test)"""





[7.94329388e-01 1.37763892e-01 3.64962731e-02 1.76884137e-02
 5.46143862e-03 2.87145924e-03 1.73575786e-03 1.35025321e-03
 9.02570708e-04 5.13383803e-04 3.22294616e-04 2.05382679e-04
 1.17692248e-04 8.74557186e-05 3.53271195e-05 2.39504513e-05
 2.25508996e-05 1.57917238e-05 9.51647350e-06 7.66650450e-06
 6.68834295e-06 5.22368885e-06 4.79936975e-06 3.64336843e-06
 3.61176853e-06 2.55391495e-06 1.72841571e-06 1.56339699e-06
 1.42142976e-06 1.11806988e-06 1.02962326e-06 8.04270468e-07
 7.37543243e-07 5.63729580e-07 4.90565254e-07 4.73898170e-07
 3.95077083e-07 3.63822558e-07 3.12209066e-07 2.65579707e-07
 2.11964270e-07 1.75596309e-07 1.50199028e-07 1.40408545e-07
 1.27552279e-07 1.13037616e-07 8.97446388e-08 7.86864314e-08
 7.45024511e-08 6.52251197e-08 6.32582635e-08 6.08917398e-08
 4.85563502e-08 3.80516105e-08 3.71194282e-08 3.18605243e-08
 3.12782110e-08 2.90279611e-08 2.71484244e-08 2.32918717e-08
 2.23662975e-08 2.05981245e-08 1.92419079e-08 1.51798942e-08
 1.21804115e-08 1.150928

ValueError: ignored

# MÉTODOS AVANZADOS

## SVM

## RANDOM FORESTS

# SELECCIÓN Y EVALUACIÓN DEL MEJOR MÉTODO